In [16]:
# 필요한 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import numpy as np
import requests
from bs4 import BeautifulSoup

In [18]:
#크롬드라이버 버전 확인
!chromedriver --version

ChromeDriver 129.0.6614.0 (d6ad6270cf0eb2ff492cf274ce38e6e62369626c-refs/branch-heads/6614@{#1})


In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# ChromeDriver 경로 설정
chrome_service = Service("chromedriver.exe")  # 다운로드한 ChromeDriver 경로

# Chrome 웹 드라이버 실행
browser = webdriver.Chrome(service=chrome_service)

# 암묵적 대기 설정
browser.implicitly_wait(10)

# 웹 페이지 열기
browser.get('https://kin.naver.com/people/expert/index.naver?type=DOCTOR&edirId=11&orgId=0&sort=name')


## 수집방법

1. 각 의사마다 url을 get 해서 리스트에 저장
2. url에 들어가서 전체 답변 페이지에 접속
3. 답변의 제목, 날짜, 내용, 답변 크롤링 -> 뒤로가기 -> 다음 답변 크롤링
4. 페이지 이동
5. 의사 별로 2-4번 반복

## 의사 별 전문가 답변 링크 추출

In [3]:
#10개 페이지에서 의사 개인 링크 추출
links=[]

for i in range(10):
    browser.get(f'https://kin.naver.com/people/expert/index.naver?orgId=0&sort=name&edirId=11&type=DOCTOR&page={i+1}')
    
    # 특정 href를 포함하는 <a> 태그 찾기
    elements = browser.find_elements(By.CSS_SELECTOR, 'a[href^="/userinfo/expert/index.naver?u="]')
    
    # href 속성 추출
    for element in elements:
        href_value = element.get_attribute('href')
        links.append(href_value)

print(len(links))

220


In [18]:
links=[]

for i in [11,12]:
    browser.get(f'https://kin.naver.com/people/expert/index.naver?orgId=0&sort=name&edirId=11&type=DOCTOR&page={i}')
    
    # 특정 href를 포함하는 <a> 태그 찾기
    elements = browser.find_elements(By.CSS_SELECTOR, 'a[href^="/userinfo/expert/index.naver?u="]')
    
    # href 속성 추출
    for element in elements:
        href_value = element.get_attribute('href')
        links.append(href_value)

In [4]:
# 중복을 제거한 값만을 담을 리스트
doctor_links= []

# 중복 제거 처리 (순서 유지)
for item in links:
    if item not in doctor_links:
        doctor_links.append(item)

# 결과 출력
print(doctor_links[:5])

['https://kin.naver.com/userinfo/expert/index.naver?u=27c0TFWFaBWEV28qvF5FAu%2FyZq92PLpBkUj9tfL4EI0%3D', 'https://kin.naver.com/userinfo/expert/index.naver?u=cfud%2BUf%2FtqvsNcVOn9GVTJ4FI%2FUKt%2BvWegYFmIEznr4%3D', 'https://kin.naver.com/userinfo/expert/index.naver?u=SvIUE03Z6u4GifVVCAvglJAhWRaLDtrESljepoVPfT0%3D', 'https://kin.naver.com/userinfo/expert/index.naver?u=%2B28rxr4a6v53CC9WL3IcaOr28A8DZLBlTKevxS2yHkA%3D', 'https://kin.naver.com/userinfo/expert/index.naver?u=QbPYSQHbEVHsy13frYO7Wkq9LJla1qotlZkVSwYDsqk%3D']


## 각 의사 페이지마다 답변 크롤링

In [20]:
# 한 페이지당 20개의 포스트 탐색 -> 엔터키 추가!
from selenium.webdriver.common.keys import Keys

#20개 포스트 크롤링하는 함수 생성
def page_scrapper():
    for j in range(1, 21): 
        try:
            post = browser.find_element(By.CSS_SELECTOR, '#au_board_list > tr:nth-child(' + str(j) + ') > td.title > a')
            post.click()

            # 경고창이 있는지 확인
            try:
                alert = Alert(browser)
                alert.accept()
                continue  # 다음 포스트로 이동

            except NoAlertPresentException:
                # 경고창이 없는 경우 크롤링 수행
                title = browser.find_element(By.CSS_SELECTOR, '#content > div.endContentLeft._endContentLeft > div.contentArea._contentWrap > div.endTitleSection').text
                date = browser.find_element(By.CSS_SELECTOR, '#content > div.endContentLeft._endContentLeft > div.contentArea._contentWrap > div.userInfo.userInfo__bullet > span:nth-child(3)').text
                question = browser.find_element(By.CSS_SELECTOR, '#content > div.endContentLeft._endContentLeft > div.contentArea._contentWrap > div.questionDetail').text
                answer = browser.find_element(By.CSS_SELECTOR, 'div.answerDetail._endContents._endContentsText').text

                # 결과 리스트에 저장
                pages.append([title, date, question, answer])
                
                browser.back()

        except NoSuchElementException:
            continue

        finally:
            # 알림 창이 없는 경우에도 뒤로 가기
            #browser.back()
            time.sleep(1)  # 페이지 로딩 시간 대기

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoAlertPresentException, NoSuchElementException
import time

chrome_service = Service("chromedriver.exe")
browser = webdriver.Chrome(service=chrome_service)
browser.implicitly_wait(10)

# 내용 담을 리스트
pages = []

for link in doctor_links:      
    
    browser.maximize_window()
    print('open: ',link)
    browser.get(link)
    time.sleep(1)
    
    more = browser.find_element(By.CSS_SELECTOR, '#content > h4 > span.more > a')
    more.click()
    

    for i in range(1, 11):
        try:
            page_selector = f'#content > div.paginate._default_pager > a:nth-child({i})'
            page = browser.find_element(By.CSS_SELECTOR, page_selector)
            page.click()
            time.sleep(1)
            page_scrapper()
        except NoSuchElementException:
            break  # 페이지 번호 버튼을 찾을 수 없을 때 종료

    #다음페이지 11~ 크롤링
    while True:
        try:
            # 다음 페이지 세트로 넘어가기 위해 '다음' 버튼 클릭
            next_button = browser.find_element(By.CSS_SELECTOR, '#content > div.paginate._default_pager > a.next')
            next_button.click()

            for i in range(2, 12):
                try:
                    # 페이지 번호 버튼 선택
                    page_selector = f'#content > div.paginate._default_pager > a:nth-child({i})'
                    page = browser.find_element(By.CSS_SELECTOR, page_selector)
                    page.click()
                    time.sleep(1)
                    page_scrapper()
                except NoSuchElementException:
                    break  # 페이지 번호 버튼을 찾을 수 없을 때 종료
        except:
            for i in range(2, 12):
                try:
                    # 페이지 번호 버튼 선택
                    page_selector = f'#content > div.paginate._default_pager > a:nth-child({i})'
                    page = browser.find_element(By.CSS_SELECTOR, page_selector)
                    page.click()
                    time.sleep(1)
                    page_scrapper()
                except NoSuchElementException:
                    break  # 페이지 번호 버튼을 찾을 수 없을 때 종료
            break
        #finally:
         #   print('done: ',link)
          #  break
        

open:  https://kin.naver.com/userinfo/expert/index.naver?u=NhKqTckrGsUpPQX5COuuNYAl4IcoSSmJPPgWi0uB7Xk%3D
open:  https://kin.naver.com/userinfo/expert/index.naver?u=tYuqffLqOiT3XXFR7EcoqF5jEJqX8VYYEvjH6cUYo1U%3D
open:  https://kin.naver.com/userinfo/expert/index.naver?u=m6V%2BptYKBpGQrHfBYvzaxAACCmWzKqpDhHsUBMZ3Opw%3D
open:  https://kin.naver.com/userinfo/expert/index.naver?u=Xoz0Wxfur%2F7hpL0qDWHUkTKvyz9Gzz7FiICyDncYQIo%3D
open:  https://kin.naver.com/userinfo/expert/index.naver?u=muRAqD6M6AZv1lFnK9HSVYdF93jaiT2ua%2FfGkO9evRU%3D
open:  https://kin.naver.com/userinfo/expert/index.naver?u=lyPNo0UVHZA1hBIQIH2yE7pfo7OqRA%2BjFoAsPDgCaxY%3D
open:  https://kin.naver.com/userinfo/expert/index.naver?u=vj9sWMlCerG%2BDT8H2SCiy2JKBBPAH95tW62%2F1Hxe6dM%3D


In [25]:
import pandas as pd

p=pd.DataFrame(pages, columns=['title','date','question','answer'])
p

,title,date,question,answer
0,질문\n모든게 다 빨라지는 느낌 머릿속 생각이나 행동을 빨리감기한 느낌,작성일\n2022.04.19,어렸을 때부터 간혹 이런 적이 있는데요.\n그냥 모든게 다 빨라져요. 지금 타이핑하...,안녕하세요. 하이닥-네이버 지식iN 상담의 권순모 입니다.\n덧글은 확인 및 답변이...
1,질문\n정신과 약,작성일\n2022.04.20,"항울제랑 항불안제 처방 받았는데 무슨 약일까요,",안녕하세요. 하이닥-네이버 지식iN 상담의 권순모 입니다.\n덧글은 확인 및 답변이...
2,질문\n뇌파검사 질문,작성일\n2020.12.21,항우울증 약 먹고있는데 뇌파검사 전에 약 안먹고 가야하나요? 아님 약 먹고 가도 되나요?,안녕하세요. 하이닥-네이버 지식iN 상담의 권순모 입니다.\n덧글은 확인 및 답변이...
3,질문\n신경안정제,작성일\n2020.12.09,화가 나면 감정조절이 잘안되고 불안장애가 심해서 무슨일이 생길것같고 벌벌 떨리고 그...,안녕하세요. 하이닥-네이버 지식iN 상담의 권순모 입니다.\n덧글은 확인 및 답변이...
4,질문\n스트레스 받으면,작성일\n2020.12.12,제가 요새 스트레스 받을 일이 좀 많았었는데요\n공부에도 집중을 못하겠고 억지로 집...,안녕하세요. 하이닥-네이버 지식iN 상담의 권순모 입니다.\n덧글은 확인 및 답변이...
...,...,...,...,...
2112,질문\n이건 무슨 강박증이나 병인가요?? 내공 100 제발요 ㅠㅠ,작성일\n2019.07.14,막 시계추 같은거 왔다 갔다 하는거랑 사소하게 바람불면 시소같은거 앞 뒤로 흔들리잖...,안녕하세요. 하이닥-네이버 지식iN 상담의 권순모 입니다.\n영상 답변 드리겠습니다...
2113,질문\n안녕하세요 저는 20대 남성입니다...,작성일\n2019.07.14,안녕하세요 저는 20대 남성입니다\n10대후반부터 머리가 멍하고 정신이 없는 증상이...,안녕하세요. 하이닥-네이버 지식iN 상담의 권순모 입니다.\n영상 답변 드리겠습니다...
2114,질문\n외상후 스트레스 심리치료방법,작성일\n2019.07.14,최근 일어난 극심한 충격과 공포심이 들 사건이 생기면서부터\n타인을 향해\n또는 나...,안녕하세요. 하이닥-네이버 지식iN 상담의 권순모 입니다.\n영상 답변 드리겠습니다...
2115,질문\n제가 대인기피증과 우울증을 1년 동안...,작성일\n2019.07.15,제가 대인기피증과 우울증을 1년 동안 앓고 있어서 많이 스트레스를 받는 바람에 신경...,안녕하세요. 하이닥-네이버 지식iN 상담의 권순모 입니다.\n영상 답변 드리겠습니다...


In [26]:
p.to_csv('doctor_links_again1.csv')